In [34]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [35]:
from sentence_transformers import SentenceTransformer
# from sentence_transformers import models

bert_path = "../../corpus/pretrained/sbert_unclear1"
sbert = SentenceTransformer(bert_path)

[2266] 2021-12-09 10:20:53,896 Info sentence_transformers.SentenceTransformer :Load pretrained SentenceTransformer: ../../corpus/pretrained/sbert_unclear1
[2266] 2021-12-09 10:20:55,312 Info sentence_transformers.SentenceTransformer :Use pytorch device: cuda


In [36]:
from sklearn import svm
# clf = AdaBoostClassifier()
smodelM = DataManager("../models/context2/")
smodel_name = "sclf_context.pickle"
sclf = smodelM.load_data(smodel_name)

success load : ../models/context2/sclf_context.pickle


In [37]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)

def text2feature(text):
    vector = text2vec(text)
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [38]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs_ = read_conv(path, datalist)

In [39]:
def make_Xy_svm(convs):
    errors = ["Topic transition error", "Lack of information", "Unclear intention"]
    errors = errors[1:]
    print(errors)
    X = []
    y = []
    for conv in convs:
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            if not ut.is_system() :
                # システムはエラー
                if conv[i+1].is_exist_error() and not conv[i+1].is_utt_level_error():
                    X.append( [ut.utt, conv[i+1].utt]  )
                    if conv[i+1].is_error_included(errors):
                        y.append(1)
                    else:
                        y.append(0)
    
    return X, y

In [40]:
X_, y_ = make_Xy_svm(convs_)

['Lack of information', 'Unclear intention']


In [41]:
X = [ text2feature(x) for x in X_]
print(len(X))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

976


In [42]:
y_pred = sclf.predict(X)

In [43]:
y = y_

In [45]:
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[155 289]
 [151 381]]
accuracy =  0.5491803278688525
precision =  0.5686567164179105
recall =  0.7161654135338346
f1 score =  0.6339434276206323


- sentence BERT のみ

    confusion matrix = 
    [[454 228]
    [191 513]]
    accuracy =  0.6976911976911977
    precision =  0.6923076923076923
    recall =  0.7286931818181818
    f1 score =  0.7100346020761246

- 発話レベルを除去

    confusion matrix = 
    [[115 157]
    [191 513]]
    accuracy =  0.6434426229508197
    precision =  0.7656716417910447
    recall =  0.7286931818181818
    f1 score =  0.7467248908296942